In [ ]:
import spin_simulation as ss
import numpy as np
import scipy.linalg as spla
import scipy.stats as stats
import importlib
from tqdm.auto import trange
import matplotlib.pyplot as plt

In [ ]:
N = 4
dim = 2**N

# pulse = .25e-6    # duration of pulse
# delay = 3e-6      # duration of delay
coupling = 2*np.pi * 5e3    # coupling strength
delta = 2*np.pi * 500       # chemical shift strength (for identical spins)

(x,y,z) = (ss.x, ss.y, ss.z)
(X,Y,Z) = ss.get_total_spin(N, dim)

Hdip, Hint = ss.getAllH(N, dim, coupling, delta)
HWHH0 = ss.get_H_WHH_0(N, dim, delta)

Compare the fidelity between UWHH0 and UHint after 5 microseconds.

In [ ]:
numSamples = 20
numTimesteps = 50
timeStep = 2
rewardMat = np.zeros((numSamples,numTimesteps))
fidelityMat = np.zeros_like(rewardMat)
t = np.zeros((numTimesteps))
for i in range(numSamples):
    Hdip, Hint = ss.getAllH(N, dim, coupling, delta)
    HWHH0 = ss.get_H_WHH_0(N, dim, delta)
    
    UWHH0 = ss.get_propagator(HWHH0, 1e-6)
    UHint = ss.get_propagator(Hint, 1e-6)
#     print(f'fidelity for 1 microsecond: {-np.log(1-ss.fidelity(UWHH0, UHint))}')
#     U1 = np.copy(UWHH0)
#     U2 = np.copy(UHint)
    for j in range(numTimesteps):
        fidelityMat[i,j] = ss.fidelity(np.linalg.matrix_power(UWHH0, timeStep*(j+1)),
                                       np.linalg.matrix_power(UHint, timeStep*(j+1)))
#         fidelityMat[i,j] = ss.fidelity(U1, U2)
        rewardMat[i,j] = -np.log10(1-fidelityMat[i,j] + 1e-100)
        t[j] = timeStep*(j+1)
#         t[j] = 2**(j+1)
#         U1 = U1 @ U1
#         U2 = U2 @ U2

meanF = np.mean(fidelityMat, axis=0)
stdF = np.std(fidelityMat, axis=0)
meanR = np.mean(rewardMat, axis=0)
stdR = np.std(rewardMat, axis=0)

In [ ]:
%matplotlib inline

plt.errorbar(t, meanF, stdF)
# plt.plot(fidelityMat.T)

In [ ]:
%matplotlib inline

plt.errorbar(t, meanR, stdR)
plt.yscale('log')
plt.xscale('log')
plt.grid(True, which='both')

# plt.plot(t, meanR**(1/-0.5))

Trying to understand the relationship between rewards and time. They exhibit an inverse relationship, but unclear whether it's a power law or exponential relationship. The reward behavior is different for small vs large $t$.

In [ ]:
stats.linregress(np.log(t[:]), np.log(meanR[:]))

Some quick maths, it takes about 250 seconds to make 40x40x40 = 64,000 calculations. That's 256 calculations per second.

In [ ]:
numSamples = 28
numDipCoupling = 5
pulses = np.geomspace(1e-7,1e-5, numSamples)
delays = np.geomspace(1e-6,1e-4, numSamples)

fMat = np.zeros((numSamples, numSamples))
rewardMat = np.zeros((numSamples, numSamples))
for i in trange(numSamples):
    p = pulses[i]
    for j in range(numSamples):
        d = delays[j]
        fTot = 0.0
        rTot = 0.0
        for k in range(numDipCoupling):
            _, Hint = ss.getAllH(N, dim, coupling, delta)
            UWHH = ss.get_U_WHH(Hint, d, p, X, Y)
            UWHH0 = ss.get_propagator(HWHH0, 6*d + 4*p)
            numCycles = np.ceil(1e-4 / (6*d+4*p))
            f = ss.fidelity(np.linalg.matrix_power(UWHH, int(numCycles)),
                            np.linalg.matrix_power(UWHH0, int(numCycles)))
            fTot += f
            rTot += -np.log10(1 - f + 1e-100)
        fMat[i,j] = fTot / numDipCoupling
        rewardMat[i,j] = rTot / numDipCoupling

In [ ]:
%matplotlib inline

X1,Y1 = np.meshgrid(pulses, delays)

plt.pcolormesh(X1, Y1, rewardMat.T)
plt.title('Rewards vs pulse, delay durations')
plt.colorbar()
plt.xlabel('Pulse length')
plt.ylabel('Delay length')
plt.xscale('log')
plt.yscale('log')